In [1]:
import numpy as np
import pandas as pd

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')



In [2]:
def init_weights(self):
    for module in modules():
        if (isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear)):
            nn.init.xavier_uniform(module.weight)
            module.bias.data.fill_(0.01)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        conv1 = nn.Conv2d(1, 32, 3, 1, 2)
        conv2 = nn.Conv2d(32, 32, 3, 1, 2)
        conv2_drop = nn.Dropout2d(.5)
        bnm1 = nn.BatchNorm1d(288, momentum=0.1)    
        fc1 = nn.Linear(288, 128)
        bnm2 = nn.BatchNorm1d(128, momentum=0.1)
        fc2 = nn.Linear(128, 64)
        bnm3 = nn.BatchNorm1d(64, momentum=0.1)
        fc3 = nn.Linear(64, 64)
        fc4 = nn.Linear(64, 2)
    

    def forward(self, x):
        x = F.max_pool2d(conv1(x), 3)
        x = F.max_pool2d(conv2_drop(conv2(x)), 3)
        x = x.view(x.size(0), -1)
        x = F.relu(F.dropout(fc1(bnm1(x)), 0.4))
        x = F.relu(F.dropout(fc2(bnm2(x)), 0.4))
        x = F.relu(F.dropout(fc3(bnm3(x)), 0.4))
        x = fc4(x)
        return F.log_softmax(x)

In [11]:
X_train = pd.read_pickle('../data/05_adjmat_data/X_train.pkl')
y_train = pd.read_pickle('../data/05_adjmat_data/y_train.pkl')
X_test = pd.read_pickle('../data/05_adjmat_data/X_test.pkl')
y_test = pd.read_pickle('../data/05_adjmat_data/y_test.pkl')

In [4]:
dims = X_train['Matrices'][0].shape

X_train, X_test = [torch.FloatTensor(x) for x in \
    [X_train['Matrices'], X_test['Matrices']]]
y_train, y_test = [torch.LongTensor(x) for x in [y_train.astype(int), y_test.astype(int)]]

X_train = torch.reshape(X_train, (-1, 1, dims[0], dims[1]))
X_test = torch.reshape(X_test, (-1, 1, dims[0], dims[1]))

train_loader = torch.utils.data.DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=32,
    shuffle=True
    )

val_loader = torch.utils.data.DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=32,
    shuffle=True
    )

In [5]:
def init_weights(self):
    for module in self.modules():
        if (isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear)):
            nn.init.xavier_uniform(module.weight)
            module.bias.data.fill_(0.01)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 2)
        self.conv2 = nn.Conv2d(32, 32, 3, 1, 2)
        self.conv2_drop = nn.Dropout2d(.5)
        self.bnm1 = nn.BatchNorm1d(288, momentum=0.1)    
        self.fc1 = nn.Linear(288, 128)
        self.bnm2 = nn.BatchNorm1d(128, momentum=0.1)
        self.fc2 = nn.Linear(128, 64)
        self.bnm3 = nn.BatchNorm1d(64, momentum=0.1)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 2)
    

    def forward(self, net):
        net = F.max_pool2d(self.conv1(net), 3)
        net = F.max_pool2d(self.conv2_drop(self.conv2(net)), 3)
        net = net.view(net.size(0), -1)
        net = F.relu(F.dropout(self.fc1(self.bnm1(net)), 0.4))
        net = F.relu(F.dropout(self.fc2(self.bnm2(net)), 0.4))
        net = F.relu(F.dropout(self.fc3(self.bnm3(net)), 0.4))
        net = self.fc4(net)
        return F.log_softmax(net)

In [6]:
loss_per_epoch_train = []
loss_per_epoch_val = []

torch.manual_seed(42)
network = Net()
network.apply(init_weights)

loss_fn=nn.CrossEntropyLoss()
optimizer = optim.Adam(
    network.parameters(),
    lr=0.0001
    )

early_stop_thres = 10
early_stop_counter = 0

for epoch in range(10):

    loss_per_batch_train = []
    loss_per_batch_val = []

    for x, y in train_loader:
                    
        optimizer.zero_grad()
        pred = network(x)
        loss=loss_fn(pred, y)
        loss_per_batch_train = loss_per_batch_train + [loss.item()]
        loss.backward()
        optimizer.step()

    loss_per_epoch_train = loss_per_epoch_train + [np.array(loss_per_batch_train).mean()]

    network.eval()

    for x, y in val_loader:

        pred_val = network(x)
        loss_eval=loss_fn(pred_val, y)
        loss_per_batch_val = loss_per_batch_val + [loss_eval.item()]

    last_epoch = epoch

    last_loss = np.array(loss_per_batch_val).mean()
    if len(loss_per_epoch_val) > 0:
        max_loss = max(loss_per_epoch_val)
    else:
        max_loss = 0
    loss_per_epoch_val = loss_per_epoch_val + [last_loss]

    if last_loss > max_loss:
        early_stop_counter += 1
        if early_stop_counter == early_stop_thres:

            break

In [10]:
X_train

tensor([[[[0.0000, 0.3945, 0.3938,  ..., 0.7682, 0.8331, 0.4272],
          [0.3945, 0.0000, 0.3956,  ..., 0.7667, 0.8327, 0.4284],
          [0.3938, 0.3956, 0.0000,  ..., 0.7701, 0.8336, 0.4258],
          ...,
          [0.7682, 0.7667, 0.7701,  ..., 0.0000, 0.8046, 0.8566],
          [0.8331, 0.8327, 0.8336,  ..., 0.8046, 0.0000, 0.7414],
          [0.4272, 0.4284, 0.4258,  ..., 0.8566, 0.7414, 0.0000]]],


        [[[0.0000, 0.3985, 0.3997,  ..., 0.5080, 0.3925, 0.4865],
          [0.3985, 0.0000, 0.3961,  ..., 0.5068, 0.3893, 0.4828],
          [0.3997, 0.3961, 0.0000,  ..., 0.5089, 0.3949, 0.4894],
          ...,
          [0.5080, 0.5068, 0.5089,  ..., 0.0000, 0.8331, 0.5725],
          [0.3925, 0.3893, 0.3949,  ..., 0.8331, 0.0000, 0.3801],
          [0.4865, 0.4828, 0.4894,  ..., 0.5725, 0.3801, 0.0000]]],


        [[[0.0000, 0.3992, 0.4000,  ..., 0.5386, 0.4002, 0.4197],
          [0.3992, 0.0000, 0.3953,  ..., 0.5341, 0.3964, 0.4148],
          [0.4000, 0.3953, 0.0000,  ..